In [0]:
import numpy
import pandas
import string
from keras.datasets import imdb
import re
import nltk
from nltk.stem.porter import PorterStemmer
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout


vocabulary_size = 5000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)

TranslatedData = pandas.read_csv('https://raw.githubusercontent.com/alisoltanirad/Sentiment-Analysis-Farsi-Dataset/master/TranslatedDigikalaDataset.csv', sep=',')

F_y = TranslatedData.iloc[0:719, 1].values

F_X = []
for i in range(719):
    review = re.sub('[^a-zA-Z]', ' ', TranslatedData['Comment'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    F_X.append(review)


max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
#F_X = sequence.pad_sequences(F_X, maxlen=max_words)
cv = CountVectorizer(max_features=500)
F_X = cv.fit_transform(F_X).toarray()


X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.15, random_state=0)
F_X_train, F_X_test, F_y_train, F_y_test = train_test_split(F_X, F_y, test_size=0.15, random_state=0)


classifier = Sequential()
classifier.add(Embedding(vocabulary_size, 200, input_length=max_words))
classifier.add(LSTM(200))
classifier.add(Dense(1, activation='sigmoid'))

classifier.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


classifier.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=1, verbose=1)

weights1 = classifier.layers[0].get_weights()
weights2 = classifier.layers[1].get_weights()

'''
model = Sequential()
model.load_weights('cache/vgg16_weights.h5')
'''

#classifier.layers.pop()
#classifier.outputs = [classifier.layers[-1].output]
#classifier.layers[-1].outbound_nodes = []

#for layer in classifier.layers:
#    layer.trainable = False

model = Sequential()
model.add(Embedding(vocabulary_size, 200, input_length=max_words))
model.add(LSTM(200))
model.layers[0].set_weights(weights1)
model.layers[1].set_weights(weights2)

for layer in model.layers:
    layer.trainable = False

model.add(Dense(200, activation='relu'))

model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

#valid_num = 54
#F_X_valid, F_y_valid = F_X_train[:valid_num], F_y_train[:valid_num]
#F_X_train, F_y_train = F_X_train[valid_num:], F_y_train[valid_num:]

model.fit(F_X_train, F_y_train, batch_size=1, epochs=3, verbose=1)


F_y_pred = model.predict(F_X_test)
F_y_pred = (F_y_pred > 0.5)


# Confusion Matrix
cm = confusion_matrix(F_y_test, F_y_pred)
accuracy = round(((cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1]))*100,3)
print('\n')
print(accuracy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Train on 21250 samples, validate on 3750 samples
Epoch 1/1
21250/21250 [==============================] - 1282s 60ms/step - loss: 0.4860 - acc: 0.7644 - val_loss: 0.4482 - val_acc: 0.8053
Epoch 1/3
611/611 [==============================] - 36s 59ms/step - loss: 0.6848 - acc: 0.5581
Epoch 2/3
611/611 [==============================] - 35s 57ms/step - loss: 0.6829 - acc: 0.5745
Epoch 3/3
611/611 [==============================] - 35s 57ms/step - loss: 0.6831 - acc: 0.5777


60.185


In [0]:
# Importing the libraries
import numpy
#import matplotlib.pyplot as plt
import pandas
#import re   #Cleaning the texts
#import nltk
#from nltk.stem.porter import PorterStemmer
#import ssl
"""
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
from nltk.corpus import stopwords
"""
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import io
import requests
import string

# Importing the dataset
dataset = pandas.read_csv("https://raw.githubusercontent.com/alisoltanirad/Sentiment-Analysis-Farsi-Dataset/master/DigikalaSentimentAnalysisDataset.csv", sep=",")

# Cleaning texts
punctuations = set(string.punctuation)
corpus = []
for i in range(719):
    data = str(dataset['Comment'][i])
    data = ''.join((char for char in data if char not in punctuations))
    corpus.append(data)


# Creating the Bag of Words model
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[0:719, 1].values

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)


# Fitting classifier to the Training set
classifier = RandomForestClassifier(n_estimators=10, criterion='entropy')
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate Accuracy
Accuracy = ((cm[0][0] + cm[1][1]) / (cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])) * 100
print(Accuracy)

78.33333333333333
